In [14]:
# imports
import wikipedia
import pandas as pd
import re

In [4]:
wikipedia.set_lang('tt')

page = wikipedia.page('бавырсак')
page.content

'Бавырса́к (монг. Борц) — татар милли ризыгы. Камыр кисәкләрен кайнаган майда кыздырып әзерләнә.\n\n\n== Әзерләү ысулы ==\nСавытка йомырка сыталар, сөт салалар, май, шикәр комы, тоз, азрак чүпрә өстиләр (күп вакытта чүпрә салмыйлар да). Шул катнашманы шикәре, тозы һәм чүпрәсе эреп беткәнче яхшылап болгаталар. Аннары он кушып, токмач камырыннан йомшаграк итеп камыр басалар. Камырны 100—150 г лы кисәкләргә бүләләр дә бармак калынлыгында баусыман тәгәрәтәләр һәм урман чикләвеге кадәр итеп турыйлар. Кайнап торган майга салалар, болгата-болгата, кызарганчы пешерәләр.\nӘзер бавырсакны тишекле зур чүмечтә сөзәләр, мае агып беткәч, өстенә шикәр оны сибергә була. Бавырсакны чәй янына чыгаралар. Аны катык һәм сөт белән дә ашарга була. Ул юлга алу өчен дә әйбәт.\n1 кг югары сортлы онга: 10 йомырка, 130—140 г сөт, 30—35 г шикәр комы, 30 г сары май, 5 г чүпрә, тол, пешерү өчен 180—200 г май.\n\n\n== Бавырлы бавырсак ==\nӘче камыр куялар. Баскан вакытта пешереп ит тарткычтан чыгарган бавыр өстиләр. 

In [20]:
text = 'бавырсакка татарларга миллинең ризыгы камыр кисәкләрен кайнаган майда '+\
'кыздырып әзерләнә әзерләү ысулы савытка йомырка сыталар сөтсалалар май комы өстиләр'

text

'бавырсакка татарларга миллинең ризыгы камыр кисәкләрен кайнаган майда кыздырып әзерләнә әзерләү ысулы савытка йомырка сыталар сөтсалалар май комы өстиләр'

In [134]:
TT_ALPHA_REGEX = r'аәбвгдеёжҗзийклмнңоөпрстуүфхһцчшщъыьэюя'


sound_pairs = {'б':'п','в':'ф','г':'к','д':'т','ж':'ш','җ':'щ','з':'с'}
deaf_pairs = {v:k for k,v in sound_pairs.items()}

def sound_deaf(intext, m1,m2):
    text = intext
    for k,v in sound_pairs.items():
        text = re.sub(k+m1+r'\b', v+m2, text)
    for k,v in deaf_pairs.items():
        text = re.sub(k+m1+r'\b', v+m2, text)
    
    return text


def reduce_affix(text, affixes,m1='',m2='_'):
#     regex = '|'.join(affixes)
#     regex = f'({regex})'
#     regex = regex+r'\b'
    reduced =text 
    for a in affixes:
        reduced = re.sub(a+m1+r'\b', m2, reduced)
    
    return reduced

figil_shart_barlik_affix = 'сам саң са сак сагыз салар'.split()
figil_shart_yuklik_affix = ['ма'+a for a in figil_barlik_affix]
figil_past_definite_affix = 'дым дем дың дең ды де дык дек дылар деләр дыгыз дегез'.split()
figil_past_indefinite_affix = 'ганмын гәнмен ганбыз гәнбез гансың гәнсең гансыз гәнсез ган гән ганнар гәннәр'.split()
figil_present_affix = 'на нә м мын мен быз без сың сең сыз сез лар ләр нәр нәр'.split()
figil_future_affix = 'ыр ер ар әр'

def process_figil(intext):
    text = intext
    text = reduce_affix(text, figil_past_definite_affix)
    text = reduce_affix(text, figil_past_indefinite_affix)
    text = reduce_affix(text, figil_shart_yuklik_affix)
    text = reduce_affix(text, figil_shart_barlik_affix)
    text = reduce_affix(text, figil_present_affix)
    return text
    
ravesh_affix = 'ып еп'.split()

def process_ravesh(intext):
    text = intext
    text = reduce_affix(text, ravesh_affix, '', '_')
    return text

# ISEM
kilesh = 'ның нең га гә ка кә ны не дан дән тан тән да дә та тә'.split()
tartim = 'ен ым ем ың ең ы е м ң сы се ыбыз ебез ыгыз егез быз без гыз гез'.split()
san = 'лар ләр нар нәр'.split()


def process_isem(intext):
    text = intext
    text = reduce_affix(text, kilesh, m1='', m2='K')
    text = reduce_affix(text, tartim, m1='K', m2 ='T')
    text = reduce_affix(text, tartim, m1='', m2 ='T')
    text = sound_deaf(text, m1='T',m2='_')
    
    text = reduce_affix(text, san, m1='T',m2='_')
    text = re.sub('[KT]','_',text)
    return text


# SIYFAT
comparative_affix = 'рак рәк'.split()
def process_siyfat(intext):
    text = intext
    text = reduce_affix(text, comparative_affix)
    return text


text = 'бавырсакка татарларга миллинең ризыгы камыр кисәкләрен кайнаган майда '+\
'кыздырып әзерләнә әзерләү ысулы савытка йомырка сытсалар сыталар сөт салалар май комы өстиләр '+\
'бардым эшләдем бардык эшләдек бардың эшләдең бардыгыз эшләдегез барды эшләде бардылар эшләделәр '+\
' '
text = 'баручыларның китабым әбием кисәгем'
###########################
text = process_ravesh(text)
text = process_isem(text)
text = process_siyfat(text)
text = process_figil(text)

text = re.sub(r'\s+', ' ', text).strip()
text = re.sub(r'_+', ' ', text).strip()
text

'баручылар  китап  әби  кисәк'